In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
task_names = [
    "COLA", 
#     "SST2",
#     "MNLI",
#     "RTE",
#     "WNLI",
#     "QQP",
#     "MRPC",
#     "STSB",
#     "QNLI",
#     "SPACY_POS",
#     "SPACY_NER",
#     "THIRD",
#     "BLEU",
]

FILENAME = "COLA_tasks_and_payloads"

In [3]:
SEED = 1

## Load previously trained model
Hint: make sure the `bert_model` is initialized correctly!

In [4]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# Create tasks and payloads
tasks, payloads = create_tasks_and_payloads(
    task_names,
    dl_kwargs={"batch_size": 16},
#     freeze_bert=True,
    bert_model='bert-large-cased'
)

Using random seed: 401221
Loading COLA Dataset



CPU times: user 25.3 s, sys: 7.74 s, total: 33 s
Wall time: 29 s


In [ ]:
tasks, payloads

([ClassificationTask(name=COLA, loss_multiplier=1.0)],
 [Payload(COLA_train: tasks=[COLA], split=train),
  Payload(COLA_valid: tasks=[COLA], split=valid),
  Payload(COLA_test: tasks=[COLA], split=test)])

In [ ]:
from metal.mmtl.metal_model import MetalModel

model = MetalModel(tasks, seed=SEED, verbose=False)

In [ ]:
import os
import torch
model_dir = '/dfs/scratch0/chami/metal/metal/mmtl/aws/output/2019_03_14_01_58_14/0/logdir/bert_large/QNLI.STSB.MRPC.QQP.RTE.MNLI.SST2.COLA.WNLI_09_15_09'
model_path = os.path.join(model_dir, 'best_model.pth')
device = torch.device(f"cuda:0")
model.load_state_dict(torch.load(model_path, map_location=device)["model"], strict=False)

#### Sanity check that task head is trained!

In [ ]:
model.score(payloads[1])

{'COLA/COLA_valid/accuracy': 0.8464491362763915,
 'COLA/COLA_valid/matthews_corr': 0.6311800125409577}

## Define slices for evaluation

In [ ]:
%%time
from metal.mmtl.glue.glue_tasks import create_tasks_and_payloads

# define slices
slice_dict = {  # A map of the slices that apply to each task
   "COLA": ["ends_with_question_mark"]
}

# Create tasks and payloads
_, payloads_slice = create_tasks_and_payloads(
    task_names,
    dl_kwargs={"batch_size": 16},
    slice_dict=slice_dict,
#     freeze_bert=True,
    bert_model='bert-large-cased'
)

Using random seed: 128037
Loading COLA Dataset



Added label_set with 8550 labels for task COLA:ends_with_question_mark to payload COLA_train.
Added label_set with 1042 labels for task COLA:ends_with_question_mark to payload COLA_valid.
Added label_set with 1063 labels for task COLA:ends_with_question_mark to payload COLA_test.
CPU times: user 18.4 s, sys: 2.47 s, total: 20.8 s
Wall time: 21.8 s


In [ ]:
payloads_slice

[Payload(COLA_train: tasks=[COLA,COLA:ends_with_question_mark], split=train),
 Payload(COLA_valid: tasks=[COLA,COLA:ends_with_question_mark], split=valid),
 Payload(COLA_test: tasks=[COLA,COLA:ends_with_question_mark], split=test)]

#### Sanity check the number of examples in the train set

In [ ]:
import numpy as np
def count_num_labels(labels):
    return np.sum(np.array(labels) != 0)

In [ ]:
dataset = payloads_slice[0].data_loader.dataset
for labelset_name, labels in dataset.labels.items():
    print(labelset_name, count_num_labels(labels))

COLA 8550
COLA:ends_with_question_mark 615


### Evaluate baseline model on the slice of interest

In [ ]:
model.score(payloads_slice[1])

Evaluating 57 / 1042 active labels


{'COLA/COLA_valid/accuracy': 0.8464491362763915,
 'COLA/COLA_valid/matthews_corr': 0.6311800125409577,
 'COLA:ends_with_question_mark/COLA_valid/accuracy': 0.7543859649122807,
 'COLA:ends_with_question_mark/COLA_valid/matthews_corr': 0.4818181818181818}

## Finetune model on slice of interest

In [ ]:
from metal.mmtl.metal_model import MetalModel

from metal.mmtl.trainer import MultitaskTrainer
trainer = MultitaskTrainer(seed=SEED)

In [ ]:
# # only finetune on the slices, not the original task
# for p in payloads_slice:
#     p.task_names.remove('COLA')
# payloads_slice

NOTE: We are training on a different set of payloads than we initialized the model with.

In [ ]:
trainer.train_model(
    model,
    payloads_slice,
    checkpoint_metric="COLA/COLA_valid/matthews_corr",
    checkpoint_metric_mode="max",
    writer="tensorboard",
    optimizer="adamax",
    lr=3e-5,
    l2=1e-3,
    log_every=0.05, 
    score_every=0.1,
    n_epochs=10,
    progress_bar=True
)

Adding missing slice heads to train {'COLA', 'COLA:ends_with_question_mark'}
Beginning train loop.
Expecting a total of approximately 8560 examples and 535 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/vschen/metal-mmtl/logs/2019_03_17/14_38_24/config.json


[0.05 epo]: COLA:[train/loss=3.12e-01] COLA:ends_with_question_mark:[train/loss=3.99e-01] model:[train/loss=3.17e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[0.10 epo]: COLA:[train/loss=2.59e-01, COLA_valid/accuracy=8.34e-01, COLA_valid/matthews_corr=6.00e-01] COLA:ends_with_question_mark:[train/loss=3.13e-01, COLA_valid/accuracy=6.84e-01, COLA_valid/matthews_corr=3.01e-01] model:[train/loss=2.62e-01, train/lr=3.00e-05, valid/glue=6.00e-01]


/dfs/scratch0/vschen/metal-mmtl/metal/mmtl/trainer.py:489: UserWarning: You requested glue score but have fewer than 9 tasks. Be aware.
  warnings.warn(msg)


[0.15 epo]: COLA:[train/loss=2.76e-01] COLA:ends_with_question_mark:[train/loss=5.61e-01] model:[train/loss=2.96e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[0.20 epo]: COLA:[train/loss=2.47e-01, COLA_valid/accuracy=8.07e-01, COLA_valid/matthews_corr=5.21e-01] COLA:ends_with_question_mark:[train/loss=5.57e-01, COLA_valid/accuracy=6.49e-01, COLA_valid/matthews_corr=2.09e-01] model:[train/loss=2.69e-01, train/lr=3.00e-05, valid/glue=5.21e-01]
[0.25 epo]: COLA:[train/loss=2.54e-01] COLA:ends_with_question_mark:[train/loss=2.82e-01] model:[train/loss=2.56e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[0.30 epo]: COLA:[train/loss=2.67e-01, COLA_valid/accuracy=8.24e-01, COLA_valid/matthews_corr=5.68e-01] COLA:ends_with_question_mark:[train/loss=2.16e-02, COLA_valid/accuracy=7.19e-01, COLA_valid/matthews_corr=3.87e-01] model:[train/loss=2.51e-01, train/lr=3.00e-05, valid/glue=5.68e-01]
[0.35 epo]: COLA:[train/loss=2.53e-01] COLA:ends_with_question_mark:[train/loss=

Evaluating 57 / 1042 active labels
[1.01 epo]: COLA:[train/loss=2.37e-01, COLA_valid/accuracy=8.21e-01, COLA_valid/matthews_corr=5.59e-01] COLA:ends_with_question_mark:[train/loss=3.55e-01, COLA_valid/accuracy=7.54e-01, COLA_valid/matthews_corr=4.69e-01] model:[train/loss=2.46e-01, train/lr=3.00e-05, valid/glue=5.59e-01]
[1.06 epo]: COLA:[train/loss=2.13e-01] COLA:ends_with_question_mark:[train/loss=3.54e-01] model:[train/loss=2.22e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[1.11 epo]: COLA:[train/loss=2.67e-01, COLA_valid/accuracy=8.11e-01, COLA_valid/matthews_corr=5.31e-01] COLA:ends_with_question_mark:[train/loss=8.03e-01, COLA_valid/accuracy=7.19e-01, COLA_valid/matthews_corr=3.92e-01] model:[train/loss=3.11e-01, train/lr=3.00e-05, valid/glue=5.31e-01]
[1.16 epo]: COLA:[train/loss=2.45e-01] COLA:ends_with_question_mark:[train/loss=4.14e-02] model:[train/loss=2.33e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[1.21 epo]: COLA:[train/loss=2.22e-01, COLA_v

...

Evaluating 57 / 1042 active labels
[5.05 epo]: COLA:[train/loss=2.45e-01, COLA_valid/accuracy=8.00e-01, COLA_valid/matthews_corr=5.05e-01] COLA:ends_with_question_mark:[train/loss=1.04e-01, COLA_valid/accuracy=7.72e-01, COLA_valid/matthews_corr=5.08e-01] model:[train/loss=2.36e-01, train/lr=3.00e-05, valid/glue=5.05e-01]
[5.10 epo]: COLA:[train/loss=2.14e-01] COLA:ends_with_question_mark:[train/loss=9.79e-02] model:[train/loss=2.08e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[5.15 epo]: COLA:[train/loss=2.60e-01, COLA_valid/accuracy=8.02e-01, COLA_valid/matthews_corr=5.07e-01] COLA:ends_with_question_mark:[train/loss=2.83e-01, COLA_valid/accuracy=7.54e-01, COLA_valid/matthews_corr=4.75e-01] model:[train/loss=2.62e-01, train/lr=3.00e-05, valid/glue=5.07e-01]
[5.20 epo]: COLA:[train/loss=2.82e-01] COLA:ends_with_question_mark:[train/loss=6.10e-02] model:[train/loss=2.69e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[5.25 epo]: COLA:[train/loss=2.56e-01, COLA_v

[6.01 epo]: COLA:[train/loss=3.12e-01] COLA:ends_with_question_mark:[train/loss=1.49e-01] model:[train/loss=3.02e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[6.06 epo]: COLA:[train/loss=2.38e-01, COLA_valid/accuracy=7.74e-01, COLA_valid/matthews_corr=4.27e-01] COLA:ends_with_question_mark:[train/loss=9.77e-02, COLA_valid/accuracy=6.84e-01, COLA_valid/matthews_corr=3.02e-01] model:[train/loss=2.29e-01, train/lr=3.00e-05, valid/glue=4.27e-01]
[6.11 epo]: COLA:[train/loss=2.27e-01] COLA:ends_with_question_mark:[train/loss=1.57e-01] model:[train/loss=2.21e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[6.16 epo]: COLA:[train/loss=2.06e-01, COLA_valid/accuracy=7.86e-01, COLA_valid/matthews_corr=4.63e-01] COLA:ends_with_question_mark:[train/loss=9.86e-03, COLA_valid/accuracy=6.67e-01, COLA_valid/matthews_corr=2.50e-01] model:[train/loss=1.92e-01, train/lr=3.00e-05, valid/glue=4.63e-01]
[6.21 epo]: COLA:[train/loss=1.93e-01] COLA:ends_with_question_mark:[train/loss=

[7.01 epo]: COLA:[train/loss=3.83e-01] COLA:ends_with_question_mark:[train/loss=6.01e-01] model:[train/loss=3.97e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[7.07 epo]: COLA:[train/loss=2.81e-01, COLA_valid/accuracy=7.74e-01, COLA_valid/matthews_corr=4.32e-01] COLA:ends_with_question_mark:[train/loss=1.25e-01, COLA_valid/accuracy=6.67e-01, COLA_valid/matthews_corr=2.56e-01] model:[train/loss=2.72e-01, train/lr=3.00e-05, valid/glue=4.32e-01]
[7.12 epo]: COLA:[train/loss=2.19e-01] COLA:ends_with_question_mark:[train/loss=5.59e-02] model:[train/loss=2.07e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[7.17 epo]: COLA:[train/loss=3.49e-01, COLA_valid/accuracy=7.64e-01, COLA_valid/matthews_corr=4.10e-01] COLA:ends_with_question_mark:[train/loss=4.12e-01, COLA_valid/accuracy=6.84e-01, COLA_valid/matthews_corr=2.98e-01] model:[train/loss=3.54e-01, train/lr=3.00e-05, valid/glue=4.10e-01]
[7.22 epo]: COLA:[train/loss=2.65e-01] COLA:ends_with_question_mark:[train/loss=

[8.02 epo]: COLA:[train/loss=3.34e-01] COLA:ends_with_question_mark:[train/loss=7.68e-01] model:[train/loss=3.68e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[8.07 epo]: COLA:[train/loss=3.89e-01, COLA_valid/accuracy=7.50e-01, COLA_valid/matthews_corr=3.90e-01] COLA:ends_with_question_mark:[train/loss=3.03e-01, COLA_valid/accuracy=6.84e-01, COLA_valid/matthews_corr=3.14e-01] model:[train/loss=3.83e-01, train/lr=3.00e-05, valid/glue=3.90e-01]
[8.13 epo]: COLA:[train/loss=3.01e-01] COLA:ends_with_question_mark:[train/loss=1.33e-01] model:[train/loss=2.90e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[8.18 epo]: COLA:[train/loss=4.65e-01, COLA_valid/accuracy=7.40e-01, COLA_valid/matthews_corr=3.18e-01] COLA:ends_with_question_mark:[train/loss=5.18e-01, COLA_valid/accuracy=6.67e-01, COLA_valid/matthews_corr=2.50e-01] model:[train/loss=4.69e-01, train/lr=3.00e-05, valid/glue=3.18e-01]
[8.23 epo]: COLA:[train/loss=3.59e-01] COLA:ends_with_question_mark:[train/loss=

[9.03 epo]: COLA:[train/loss=3.54e-01] COLA:ends_with_question_mark:[train/loss=2.47e-01] model:[train/loss=3.46e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[9.08 epo]: COLA:[train/loss=4.17e-01, COLA_valid/accuracy=7.32e-01, COLA_valid/matthews_corr=2.95e-01] COLA:ends_with_question_mark:[train/loss=3.56e-01, COLA_valid/accuracy=6.14e-01, COLA_valid/matthews_corr=1.08e-01] model:[train/loss=4.13e-01, train/lr=3.00e-05, valid/glue=2.95e-01]
[9.13 epo]: COLA:[train/loss=3.72e-01] COLA:ends_with_question_mark:[train/loss=5.00e-01] model:[train/loss=3.79e-01, train/lr=3.00e-05]
Evaluating 57 / 1042 active labels
[9.19 epo]: COLA:[train/loss=4.32e-01, COLA_valid/accuracy=7.33e-01, COLA_valid/matthews_corr=3.08e-01] COLA:ends_with_question_mark:[train/loss=3.35e-01, COLA_valid/accuracy=6.49e-01, COLA_valid/matthews_corr=2.03e-01] model:[train/loss=4.26e-01, train/lr=3.00e-05, valid/glue=3.08e-01]


#### Did we improve?

In [ ]:
from metal.mmtl.metal_model import MetalModel
model.score(payloads_slice[1])